# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

pd.options.display.max_rows = 100

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.8+ MB


In [26]:
data_by_user = data.groupby(['user', 'country', 'sex'])['artist'].apply(lambda x: ','.join(x).lower().strip()).reset_index()
data_by_user

,user,country,sex,artist
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,..."
1,3,United States,m,"devendra banhart,boards of canada,cocorosie,ap..."
2,4,United Kingdom,m,"tv on the radio,tool,kyuss,dj shadow,air,a tri..."
3,5,Finland,m,"dream theater,ac/dc,metallica,iron maiden,bob ..."
4,6,Portugal,m,"lily allen,kanye west,sigur rós,pink floyd,ste..."
...,...,...,...,...
14995,19713,Ukraine,m,"armin van buuren,above & beyond,atb,ferry cors..."
14996,19714,United Kingdom,m,"misfits,type o negative,arch enemy,red hot chi..."
14997,19715,United Kingdom,m,"abba,james blunt,jason mraz,amy winehouse,quee..."
14998,19717,Brazil,m,"marilyn manson,beyoncé,madonna,t.a.t.u.,katy p..."


In [15]:
data_by_user_ger_neth = data_by_user[data_by_user['country'].isin(['Germany','Netherlands'])]
data_by_user_ger_neth.head()

,user,country,sex,artist
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,..."
19,31,Netherlands,f,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
21,33,Germany,f,"death cab for cutie,tegan and sara,kimya dawso..."
29,42,Germany,f,"soundtrack,groove coverage,avril lavigne,the r..."
37,51,Germany,f,"kate nash,arctic monkeys,lykke li,adele,tegan ..."


In [16]:
#dummy encoding
dummy_data_by_user_ger_neth = data_by_user_ger_neth['artist'].str.get_dummies(',')
data_ger_neth = pd.concat([data_by_user_ger_neth, dummy_data_by_user_ger_neth], axis = 1)
data_ger_neth.shape

(1724, 1006)

In [7]:
data_ger_neth.columns

Index(['user', 'country', 'artist',
       '...and you will know us by the trail of dead', '2pac', '3 doors down',
       '30 seconds to mars', '311', '36 crazyfists', '44',
       ...
       'wilco', 'within temptation', 'wolfgang amadeus mozart', 'wu-tang clan',
       'yann tiersen', 'yeah yeah yeahs', 'yellowcard', 'yo la tengo',
       'zero 7', 'édith piaf'],
      dtype='object', length=1005)

In [8]:
#pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [8]:
frequent_itemsets = apriori(data_ger_neth.drop(columns=['user','country', 'sex', 'artist']), 
                            min_support=0.015, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.017981,(2pac)
1,0.057425,(3 doors down)
2,0.041183,(30 seconds to mars)
3,0.019722,(36 crazyfists)
4,0.020302,(50 cent)
...,...,...
997,0.016821,"(rammstein, koяn, system of a down)"
998,0.015081,"(slipknot, koяn, system of a down)"
999,0.016241,"(rammstein, system of a down, metallica)"
1000,0.015081,"(rammstein, slipknot, system of a down)"


In [91]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.45)
#rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.029002,0.048144,0.015661,0.540000,11.216386,0.014265,2.069252
1,(goldfrapp),(air),0.037703,0.087007,0.017401,0.461538,5.304615,0.014121,1.695559
2,(anti-flag),(rise against),0.024942,0.070186,0.016821,0.674419,9.609072,0.015071,2.855858
3,(maxïmo park),(arctic monkeys),0.052784,0.099768,0.026102,0.494505,4.956555,0.020836,1.780894
4,(the last shadow puppets),(arctic monkeys),0.024362,0.099768,0.015081,0.619048,6.204873,0.012651,2.363109
5,(killswitch engage),(as i lay dying),0.034223,0.038283,0.016821,0.491525,12.839240,0.015511,1.891377
6,(beatsteaks),(die ärzte),0.069606,0.143271,0.031323,0.450000,3.140891,0.021350,1.557688
7,(editors),(bloc party),0.042343,0.086427,0.019722,0.465753,5.388986,0.016062,1.710021
8,(bob dylan),(the beatles),0.045824,0.123550,0.020882,0.455696,3.688358,0.015220,1.610222
9,(breaking benjamin),(linkin park),0.035963,0.142111,0.016821,0.467742,3.291376,0.011711,1.611791


In [92]:
# support - показатель «частотности»
# confidence -  показатель того, как часто наше правило срабатывает для всего датасета.
# lift - 
# leverage - отношение «зависимости» items к их «независимости»
# conviction — это «частотность ошибок»  правила
rules[ 
       (rules['antecedent support'] > 0.03) &
       (rules['lift'] > 10) &
       (rules['support'] > 0.015) 
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(killswitch engage),(as i lay dying),0.034223,0.038283,0.016821,0.491525,12.83924,0.015511,1.891377


1) Большое значение lift значит, что эти две группы слушают вместе намного чаще, чем слушают только одну группу killswitch engage, поэтому рекомендация as i lay dying, 
   когда человек случашет killswitch engage будет эффективной. 

In [10]:
frequent_itemsets2 = fpgrowth(data_ger_neth.drop(columns=['user','country','artist']), 
                            min_support=0.015, use_colnames=True)
frequent_itemsets2

,support,itemsets
0,0.154872,(red hot chili peppers)
1,0.104988,(the killers)
2,0.099768,(jack johnson)
3,0.048144,(the rolling stones)
4,0.044664,(schandmaul)
...,...,...
997,0.020302,"(the killers, editors)"
998,0.019722,"(bloc party, editors)"
999,0.016241,"(the kooks, editors)"
1000,0.015081,"(interpol, editors)"


In [11]:
rules = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.45)
#rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(the rolling stones),(the beatles),0.048144,0.123550,0.022042,0.457831,3.705639,0.016094,1.616564
1,(schandmaul),(subway to sally),0.044664,0.055684,0.023782,0.532468,9.562229,0.021295,2.019786
2,(schandmaul),(in extremo),0.044664,0.052784,0.021462,0.480519,9.103468,0.019104,1.823390
3,(goldfrapp),(air),0.037703,0.087007,0.017401,0.461538,5.304615,0.014121,1.695559
4,"(coldplay, the kooks)",(arctic monkeys),0.034803,0.099768,0.016241,0.466667,4.677519,0.012769,1.687935
5,"(coldplay, arctic monkeys)",(the kooks),0.035383,0.094548,0.016241,0.459016,4.854873,0.012896,1.673715
6,"(the kooks, arctic monkeys)",(coldplay),0.035963,0.171694,0.016241,0.451613,2.630340,0.010067,1.510441
7,"(the kooks, the killers)",(arctic monkeys),0.034223,0.099768,0.016821,0.491525,4.926685,0.013407,1.770456
8,"(the kooks, arctic monkeys)",(the killers),0.035963,0.104988,0.016821,0.467742,4.455177,0.013046,1.681537
9,"(the killers, arctic monkeys)",(the kooks),0.033063,0.094548,0.016821,0.508772,5.381122,0.013695,1.843242


In [12]:
frequent_itemsets3 = fpmax(data_ger_neth.drop(columns=['user','country','artist']), 
                            min_support=0.02, use_colnames=True)
frequent_itemsets3

,support,itemsets
0,0.020302,(klaxons)
1,0.020302,(enya)
2,0.020302,(the whitest boy alive)
3,0.020302,(dido)
4,0.020302,(simon & garfunkel)
...,...,...
451,0.037123,"(linkin park, die ärzte)"
452,0.040023,"(linkin park, red hot chili peppers)"
453,0.026682,"(coldplay, die ärzte)"
454,0.040023,"(red hot chili peppers, die ärzte)"


In [13]:
rules = association_rules(frequent_itemsets3, metric="lift", min_threshold=0.5)
#rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

KeyError: "frozenset({'coldplay'})You are likely getting this error because the DataFrame is missing  antecedent and/or consequent  information. You can try using the  `support_only=True` option"

In [151]:
data_fm = data[(data.country == 'Germany') | (data.country == 'Netherlands')].groupby(['artist', 'sex']).agg({'sex': ['count']})
data_fm.columns = data_fm.columns.map(''.join)
data_fm

sexcount
artist                                       sex          
...and you will know us by the trail of dead f           7
                                             m          17
2pac                                         f           2
                                             m          29
3 doors down                                 f          37
...                                                    ...
yo la tengo                                  m           6
zero 7                                       f           3
                                             m          16
Édith piaf                                   f           5
                                             m           3

[1909 rows x 1 columns]